In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from keras.layers import Dropout

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np
from PIL import Image

from preprocessing import preprocess
df = pd.read_csv('metadata.csv')
data_root_path = r'C:\Users\inesb\Downloads\rare_species'
train_df, test_df, val_df = preprocess(df)
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

c:\Users\inesb\anaconda3\envs\deep_learning\Lib\site-packages\PIL\Image.py:3432: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Largest image in the directory:
File Path: C:\Users\inesb\Downloads\rare_species\mollusca_cardiidae\30003931_46473744_eol-full-size-copy.jpg
Size (Width x Height): (17000, 6800)

Smallest image in the directory:
File Path: C:\Users\inesb\Downloads\rare_species\chordata_delphinidae\9484689_46559297_eol-full-size-copy.jpg
Size (Width x Height): (193, 129)
Missing images: 0
Duplicate metadata rows:
Empty DataFrame
Columns: [rare_species_id, eol_content_id, eol_page_id, kingdom, phylum, family, file_path, full_path, exists]
Index: []
Duplicate file paths:
Empty DataFrame
Columns: [rare_species_id, eol_content_id, eol_page_id, kingdom, phylum, family, file_path, full_path, exists]
Index: []
202
count    202.000000
mean      59.321782
std       54.326637
min       29.000000
25%       30.000000
50%       30.000000
75%       60.000000
max      300.000000
Name: full_path, dtype: float64
Train shape: (8387, 10)
Val shape: (1798, 10)
Test shape: (1798, 10)


TypeError: 'tuple' object is not callable

  Using cached utils-1.0.2-py2.py3-none-any.whl


ModuleNotFoundError: No module named 'utils.data_utils'

In [ ]:
data_util = DataGeneratorUtils(data_base_path)

train_generator = data_util.create_generators('train')
val_generator = data_util.create_generators('validation')

NUM_CLASSES = train_generator.num_classes
class_weights = data_util.calculate_class_weights(train_generator)

print("Classes:", NUM_CLASSES)
print("Train samples:", train_generator.samples)
print("Validation samples:", val_generator.samples)